# Using Tensorboard
-----------------------------

Neural networks are often criticized because it's difficult to understand how they learn. </br>
TensorBoard is a powerful visualization tool that allows to open the black box and go inside the model.

We'll illustrate the various uses of Tensorboard in this script.
 1. Visualize scalars, distributions, images and histograms
 2. Visualize TensorFlow model

We start by loading the necessary libraries!

In [ ]:
import tensorflow as tf
import numpy as np
import datetime

## Build MNIST Model
To illustrate the various ways we can use TensorBoard, we will reimplement the `MNIST` model from `The Introductory CNN Model` in `Chapter 8, Convolutional Neural Network`.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Padding the images by 2 pixels since in the paper input images were 32x32
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Normalize
x_train = x_train / 255
x_test = x_test/ 255

# Set model parameters
image_width = x_train[0].shape[0]
image_height = x_train[0].shape[1]
num_channels = 1 # grayscale = 1 channel

# Training and Test data variables
batch_size = 100
evaluation_size = 500
generations = 300
eval_every = 5

# Set for reproducible results
seed = 98
np.random.seed(seed)
tf.random.set_seed(seed)

# Declare the model
input_data = tf.keras.Input(dtype=tf.float32, shape=(image_width,image_height, num_channels), name="INPUT")

# First Conv-ReLU-MaxPool Layer
conv1 = tf.keras.layers.Conv2D(filters=6,
                               kernel_size=5,
                               padding='VALID',
                               activation="relu",
                               name="C1")(input_data)

max_pool1 = tf.keras.layers.MaxPool2D(pool_size=2,
                                      strides=2, 
                                      padding='SAME',
                                      name="S1")(conv1)

# Second Conv-ReLU-MaxPool Layer
conv2 = tf.keras.layers.Conv2D(filters=16,
                               kernel_size=5,
                               padding='VALID',
                               strides=1,
                               activation="relu",
                               name="C3")(max_pool1)

max_pool2 = tf.keras.layers.MaxPool2D(pool_size=2,
                                      strides=2, 
                                      padding='SAME',
                                      name="S4")(conv2)

# Flatten Layer
flatten = tf.keras.layers.Flatten(name="FLATTEN")(max_pool2)


# First Fully Connected Layer
fully_connected1 = tf.keras.layers.Dense(units=120,
                                         activation="relu",
                                         name="F5")(flatten)

# Second Fully Connected Layer
fully_connected2 = tf.keras.layers.Dense(units=84,
                                         activation="relu",
                                         name="F6")(fully_connected1)

# Final Fully Connected Layer
final_model_output = tf.keras.layers.Dense(units=10,
                                           activation="softmax",
                                           name="OUTPUT"
                                           )(fully_connected2)
    

model = tf.keras.Model(inputs= input_data, outputs=final_model_output)

Next, we will compile the model with the sparse categorical cross-entropy loss and the ADAM optimizer.
Then, we'll display the summary

In [ ]:
model.compile(
    optimizer="adam", 
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.summary()

## Add TensorBoard Callback & Fit the model

We will create a timestamped subdirectory for each runs.

In [ ]:
log_dir="logs/experiment" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

Next, we will instantiate a TensorBoard callback and pass it to the fit method. All logs during the training phase will be stored in this directory and can be able to view instantly in TensorBoard.

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                      write_images=True,
                                                      histogram_freq=1 )

model.fit(x=x_train, 
          y=y_train, 
          epochs=5,
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

## Start TensorBoard

We then start the Tensorboard application by running the command:

`$ tensorboard --logdir="logs"`

Then we navigate our browser to the following link:

`http://127.0.0.0:6006`

We can specify a different port if needed by passing for example a `--port 6007` command (for running on port 6007).

We can also start TensorBoard within the notebook through the `%tensorboard --logdir="logs"` command line 

> Note that TensorBoard will be viewable _*as*_ your program is running.

## Visualize scalars, distributions, images and histograms
We can quickly and easily visualize and compare metrics of several experiments during the model training through the TensorBoard's scalars view. By default, TensorBoard writes the metrics and losses every epoch. We can update this frequency by batch using the following argument 
`update_freq='batch'`. 

We can also visualize model weights as image with the argument `write_images=True` or display bias and weights with histograms (computation every epoch) using `histogram_freq=1 `.


Here the screenshot of the scalars view:

![scalars view](../images/01_tensorboard_scalars.png)

Here we show how to visualize weights and bias with a histogram summary.

![histogram view](../images/01_tensorboard_histograms.png)

## Visualize TensorFlow model
The TensorFlow's Graphs dashboard shows the model using different views. 

This dashboard allows to visualize the op-level graph but also the conceptual graph that displays only the Keras model without extra edges to other computation nodes.

These views allow to quickly examinate and compare our intended design and how TensorFlow understands the model structure.

Here we show how to visualize the op-level graph.

![op-level graph view](../images/01_tensorboard_graph.png)

## Use file writer

Create a FileWriter for the timestamped log directory and write the top ten images. 

In [ ]:

file_writer = tf.summary.create_file_writer(log_dir)

with file_writer.as_default():
    # Reshape the images and write the image summary
    images = np.reshape(x_train[0:10], (-1, 32, 32, 1))
    tf.summary.image("10 training data examples", images, max_outputs=10, step=0)

Here we show how to visualize the top ten images.

![image graph view](../images/01_tensorboard_image.png)
